## PAIR PROGRAMMING LIMPIEZA II

### Strings
---

**Preguntas:**

- ¿Es Australia es el sitio más peligroso y letal para estar relajada en la playa?
- ¿Cuál es el rango de edad que sufre la mayoría de los ataques?
- Independientemente de la edad, sufren los hombres más ataques que las mujeres?
- ¿En qué mes ocurren más ataques?
- ¿Cuál es la relación entre la especie y el tipo de ataque (si es fatal o no)?
- ¿Cómo han evolucionado los ataques a lo largo del tiempo?


¿Está limpia?
- age ❌      esta en formato string cuando debería ser integer y en algunos casos tenemos rangos de edad
- species ❌  es un jaleo! Debemos unificar los nombres y reducir a las especies más importantes
- country ✔️   los paises están en mayúsculas, algunos se repiten con algunos cambios
- fatal ✔️     la limpiamos en el pair de Pandas V
- year ✔️      es una columna de tipo float deberíamos convertirla a integer
- sex ✔️       la limpiamos en el pair de Pandas V
- fecha ✔️     la limpiamos en el pair de Pandas V
​

**EJERCICIO 1**

Columna de species: si exploramos esta columna en detalle nos podemos dar cuenta que tenemos muchos valores únicos y esto hace que sea muy difícil trabajar con esta columna. 

Lo que vamos a hacer es clasificar los tiburones en 5 especies diferentes, las más comunes, que incluyen el tiburón blanco (White), el tiburón tigre (Tiger ), el tiburón gris (Grey), el tiburón limón (Lemon) y el tiburón toro (Bull). El resto de las especies las incluiremos en un único grupo que podremos llamar "Unspecified". 

Los valores de las columnas son strings por lo que podremos usar regex para buscar palabras clave en cada celda y asignarlo a una de las categorías que hemos definido previamente.

Imaginemos que el valor de una celda es el siguiente
​
'White shark, 3.5 m'
​
Tendremos que buscar el patrón de regex que nos permita extraer White shark de ese string y que nos devuelva solo White shark. 

Un patrón que podriamos usar es: 
​
patron_blanco = r".*[Ww](hite|HITE).*" # esto podría ser así porque puede estar en mayúsculas o en minúsculas. 
​
De la misma forma que hemos sacado el patron para el tiburón blanco, tendremos que sacar los patrones para las otras 4 especies que queremos "encontrar". 

Tendremos que crearnos una función que aplicaremos sobre nuestra columna species_ para que nos devuelva una nueva columna con los valores clasificados en función de los patrones de regex que hayamos definido.


**EJERCICIO 2**

Columna de age: es una columna de tipo string pero debería ser de tipo integer. Además, en esta columna nos vamos a encontrar con algunos errores tipográficos, estos incluyen:

- Edad en formato string
- Edades separadas por &, or, to, >
- Edades con ?


Primero tendremos que eliminar todos esos símbolos especiales que nos aparecen. De nuevo, podremos usar regex para extraer unicamente los números que es lo que nos interesa. Usar este regex en una función para sacar solo los números.

Puede que os salga un error similar a este:
```python 
TypeError: expected string or bytes-like object
```
Para solucionar este problema, antes de nada tendréis que ejecutar este código para que no os de error:

df['nombre_columna'] = df['nombre_columna'].astype(str)

Una vez que hayáis extraido los números, os daréis cuenta que hay celdas que tienen más de una edad. Tendréis que decidir que hacer en esos casos. Os dejamos por aquí una posible opción usando un método de Pandas que os puede resultar super útil. El método .explode, 

Por último cambiad el tipo de la columna de string a integer.


**EJERCICIO 3**

Guarda el csv con las columnas limpias para seguir trabajando con este dataframe limpio.
